# <div align="center"> CV Api Test </div>

In [3]:
import requests
import json
import consul
import time

# 全局配置

In [4]:
hostname = 'gamma'
host = '10.255.20.227'
port = 8129

user = 'test'
uuid = 'abcdef'

consul_addr = host
consul_port = 8500

In [20]:
# API异步接口将过程中产出的数据同步到consul数据中心, 调用该接口可从中心获取数据 (数据中心仅供开发使用)
def query_async_data(key, trycnt=3):
    i = 0
    while i < trycnt:
        try:
            client = consul.Consul(consul_addr, consul_port)
            _, data = client.kv.get(key)
            return str(data['Value'], encoding="utf-8")
        except:
            i = i + 1
            time.sleep(2)
            print('Query nothing! try:', i)
    return 'Query nothing!!!'
    
# 格式化打印json字符串
def print_json(text):
    if isinstance(text, str):
        print(json.dumps(json.loads(text), indent=4))
    else:
        print(json.dumps(text, indent=4))

# 训练

## 1. 图像分类

In [6]:
cls_template_config = '''{
    "dataset": "cifar10", 
    "task": "cls", 
    "method": "image_classifier", 
    "data": {
        "num_classes": 10, 
        "data_dir": "/data/datasets/cv/cifar10", 
        "image_tool": "pil", 
        "input_mode": "RGB", 
        "workers": 1, 
        "mean_value": [
            124, 
            116, 
            104
        ], 
        "normalize": {
            "div_value": 255, 
            "mean": [
                0.485, 
                0.456, 
                0.406
            ], 
            "std": [
                0.229, 
                0.224, 
                0.225
            ]
        }
    }, 
    "train": {
        "batch_size": 32, 
        "aug_trans": {
            "trans_seq": [
                "random_hflip", 
                "random_border", 
                "random_crop"
            ], 
            "random_hflip": {
                "ratio": 0.5, 
                "swap_pair": [ ]
            }, 
            "random_border": {
                "ratio": 1, 
                "pad": [
                    4, 
                    4, 
                    4, 
                    4
                ], 
                "allow_outside_center": false
            }, 
            "random_crop": {
                "ratio": 1, 
                "crop_size": [
                    32, 
                    32
                ], 
                "method": "random", 
                "allow_outside_center": false
            }
        }, 
        "data_transformer": {
            "size_mode": "fix_size", 
            "input_size": [
                32, 
                32
            ], 
            "align_method": "only_pad"
        }
    }, 
    "val": {
        "batch_size": 32, 
        "aug_trans": {
            "trans_seq": [ ]
        }, 
        "data_transformer": {
            "size_mode": "fix_size", 
            "input_size": [
                32, 
                32
            ], 
            "align_method": "only_pad"
        }
    }, 
    "test": { }, 
    "details": {
        "name_seq": [
            "plane", 
            "car", 
            "bird", 
            "cat", 
            "deer", 
            "dog", 
            "frog", 
            "horse", 
            "ship", 
            "truck"
        ], 
        "color_list": [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0],
                     [85, 255, 0], [0, 255, 0], [0, 255, 85], [0, 255, 170], [0, 255, 255],
                     [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], [170, 0, 255],
                     [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 0, 170], [255, 0, 85]]
    }, 
    "network": {
        "model_name": "cls_model", 
        "backbone": "vgg19", 
        "pooled_size": [
            1, 
            1
        ],
        "checkpoints_name": "ic_vgg19_cifar10_cls", 
        "checkpoints_dir": "checkpoints/cls/cifar10"
    }, 
    "solver": {
        "lr": {
            "metric": "epoch", 
            "base_lr": 0.1, 
            "lr_policy": "multistep", 
            "step": {
                "gamma": 0.1, 
                "step_size": 30
            }, 
            "multistep": {
                "gamma": 0.1, 
                "stepvalue": [
                    150, 
                    250, 
                    350
                ]
            }
        }, 
        "optim": {
            "optim_method": "sgd", 
            "adam": {
                "betas": [
                    0.9, 
                    0.999
                ], 
                "eps": 1e-8, 
                "weight_decay": 0.0001
            }, 
            "sgd": {
                "weight_decay": 0.00004, 
                "momentum": 0.9, 
                "nesterov": false
            }
        }, 
        "display_iter": 20, 
        "save_iters": 2000, 
        "test_interval": 100, 
        "max_epoch": 360
    }, 
    "loss": {
        "loss_type": "ce_loss", 
        "loss_weights": {
            "ce_loss": {
                "ce_loss": 1
            }, 
            "soft_ce_loss": {
                "soft_ce_loss": 1
            }, 
            "mixup_ce_loss": {
                "mixup_ce_loss": 1
            }, 
            "mixup_soft_ce_loss": {
                "mixup_soft_ce_loss": 1
            }
        }, 
        "params": {
            "ce_loss": {
                "reduction": "mean", 
                "ignore_index": -1
            }, 
            "soft_ce_loss": {
                "reduction": "batchmean", 
                "label_smooth": 0.1
            }, 
            "mixup_ce_loss": {
                "reduction": "mean", 
                "ignore_index": -1
            }, 
            "mixup_soft_ce_loss": {
                "reduction": "batchmean", 
                "label_smooth": 0.1
            }
        }
    }
}'''

cls_template_config_dict = json.loads(cls_template_config)

### 1.1 cls_vgg19_cifar10

In [135]:
cls_config = cls_template_config_dict

cls_config['dataset'] = 'cifar10'
cls_config['data']['data_dir'] = '/data/datasets/cifar10'
cls_config['train']['batch_size'] = 32
cls_config['details']['name_seq'] = [
    "plane", 
    "car", 
    "bird", 
    "cat", 
    "deer", 
    "dog", 
    "frog", 
    "horse", 
    "ship", 
    "truck"
]
cls_config['solver']['max_epoch'] = 2
cls_config['solver']['optim']['optim_method'] = 'sgd'
cls_config['loss']['loss_type'] = 'ce_loss'
cls_config['network']['backbone'] = 'vgg19'
cls_config['network']['checkpoints_name'] = 'ic_vgg19_cifar10_cls'
cls_config['network']['checkpoints_dir'] = 'checkpoints'
conf = json.dumps(cls_config)

In [133]:
uuid = 'cls_vgg19_cifar10'
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success",
    "message": "train.start task cache directory: /data/users/test/abcdef"
}


异步输出样例:

```json
{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "abcdef",
    "timestamp": 1574939332062,
    "datetime": "2019-11-28 19:08:52",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 2.3058,
        "training_speed": 0.105,
        "lr": [
            0.1,
            0.1
        ],
        "validation_loss": 2.3041,
        "validation_accuracy": 0.10519999710842967,
        "validation_accuracy3": 0.3018999940156937,
        "validation_accuracy5": 0.49709998607635497
    }
}
```

In [136]:
print(query_async_data('framework/%s/%s/train.start/metrics'%(user, uuid)))

{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "abcdef",
    "timestamp": 1574942562414,
    "datetime": "2019-11-28 20:02:42",
    "metrics": {
        "training_epochs": 2,
        "training_loss": 2.3036,
        "training_speed": 0.056,
        "lr": [
            0.1,
            0.1
        ],
        "validation_loss": 2.3168,
        "validation_accuracy": 0.105,
        "validation_accuracy3": 0.295,
        "validation_accuracy5": 0.5011
    }
}


In [118]:
data = '''{
    "op":"train.stop",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success"
}


### 1.2 cls_resnet18_mnist

In [7]:
cls_config = template_config_dict

cls_config['dataset'] = 'mnist'
cls_config['data']['data_dir'] = '/data/datasets/cv/mnist'
cls_config['train']['batch_size'] = 32
cls_config['details']['name_seq'] = [
    "0", 
    "1", 
    "2", 
    "3", 
    "4", 
    "5", 
    "6", 
    "7", 
    "8", 
    "9"
]
cls_config['solver']['max_epoch'] = 2
cls_config['solver']['optim']['optim_method'] = 'adam'
cls_config['loss']['loss_type'] = 'soft_ce_loss'
cls_config['network']['backbone'] = 'resnet18'
cls_config['network']['checkpoints_name'] = 'ic_resnet18_mnist_cls'
cls_config['network']['checkpoints_dir'] = 'checkpoints'
conf = json.dumps(cls_config)

In [21]:
uuid = 'cls_resnet18_mnist'
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success",
    "message": "train.start task cache directory: /data/users/test/cls_resnet18_mnist"
}


In [24]:
print(query_async_data('framework/%s/%s/train.start/metrics'%(user, uuid), trycnt=4))

{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "cls_resnet18_mnist",
    "timestamp": 1574946168637,
    "datetime": "2019-11-28 21:02:48",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 1.49,
        "training_speed": 0.06,
        "lr": [
            0.1,
            0.1
        ],
        "validation_loss": 1.7414,
        "validation_accuracy": 0.11791666666666667,
        "validation_accuracy3": 0.32258333333333333,
        "validation_accuracy5": 0.5316666666666666
    }
}


In [25]:
data = '''{
    "op":"train.stop",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success"
}


## 目标检测

In [53]:
det_template_config = '''{
    "dataset": "voc",
    "task": "det",
    "method": "single_shot_detector",
    "data": {
      "num_classes": 21,
      "data_dir": "~/DataSet/VOC07+12_DET",
      "image_tool": "cv2",
      "input_mode": "BGR",
      "keep_difficult": false,
      "workers": 1,
      "mean_value": [104, 117, 123],
      "normalize": {
        "div_value": 1,
        "mean": [104.0, 117.0, 123.0],
        "std": [1.0, 1.0, 1.0]
      }
    },
    "train": {
      "batch_size": 16,
      "aug_trans": {
        "shuffle_trans_seq": ["random_contrast", "random_hue", "random_saturation", "random_brightness", "random_perm"],
        "trans_seq": ["random_hflip", "random_pad", "random_det_crop"],
        "random_saturation": {
          "ratio": 0.5,
          "lower": 0.5,
          "upper": 1.5
        },
        "random_hue": {
          "ratio": 0.5,
          "delta": 18
        },
        "random_contrast": {
          "ratio": 0.5,
          "lower": 0.5,
          "upper": 1.5
        },
        "random_pad": {
          "ratio": 0.6,
          "up_scale_range": [1.0, 4.0]
        },
        "random_brightness": {
          "ratio": 0.5,
          "shift_value": 32
        },
        "random_perm": {
          "ratio": 0.5
        },
        "random_hflip": {
          "ratio": 0.5,
          "swap_pair": []
        },
        "random_det_crop":{
          "ratio": 1.0
        }
      },
      "data_transformer": {
        "size_mode": "fix_size",
        "input_size": [300, 300],
        "align_method": "only_scale"
      }
    },
    "val": {
      "batch_size": 16,
      "use_07_metric": true,
      "aug_trans": {
        "trans_seq": []
      },
      "data_transformer": {
        "size_mode": "fix_size",
        "input_size": [300, 300],
        "align_method": "only_scale"
      }
    },
    "test": {
      "batch_size": 16,
      "aug_trans": {
        "trans_seq": []
      },
      "data_transformer": {
        "size_mode": "fix_size",
        "input_size": [300, 300],
        "align_method": "only_scale"
      }
    },
    "anchor": {
      "anchor_method": "ssd",
      "iou_threshold": 0.5,
      "num_anchor_list": [4, 6, 6, 6, 4, 4],
      "feature_maps_wh": [[38, 38], [19, 19], [10, 10], [5, 5], [3, 3], [1, 1]],
      "cur_anchor_sizes": [30, 60, 111, 162, 213, 264, 315],
      "aspect_ratio_list": [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
    },
    "details": {
      "color_list": [[255, 170, 30], [0, 0, 70], [244, 35, 232]],
      "name_id_dict": {
        "aeroplane": 1, "bicycle": 2, "bird": 3, "boat": 4, "bottle": 5, "bus": 6, "car": 7,
        "cat": 8, "chair": 9, "cow": 10, "diningtable": 11, "dog": 12, "horse": 13, "motorbike": 14,
        "person": 15, "pottedplant": 16, "sheep": 17, "sofa": 18, "train": 19, "tvmonitor": 20
      },
      "name_seq": ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair",
                   "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant",
                   "sheep", "sofa", "train", "tvmonitor"]
    },
    "network":{
      "backbone": "vgg16",
      "model_name": "vgg300_ssd",
      "num_feature_list": [512, 1024, 512, 256, 256, 256],
      "stride_list": [8, 16, 30, 60, 100, 300],
      "head_index_list": [0, 1, 2, 3, 4, 5],
      "checkpoints_name": "ssd_vgg300_voc_det",
      "checkpoints_dir": "checkpoints"
    },
    "solver": {
      "lr": {
        "metric": "epoch",
        "is_warm": true,
        "warm": {
          "warm_iters": 1000,
          "power": 1.0,
          "freeze_backbone": false
        },
        "base_lr": 0.001,
        "lr_policy": "multistep",
        "multistep": {
          "gamma": 0.1,
          "stepvalue": [156, 195, 234]
        }
      },
      "optim": {
        "optim_method": "sgd",
        "adam": {
          "betas": [0.9, 0.999],
          "eps": 1e-08,
          "weight_decay": 0.0001
        },
        "sgd":{
          "weight_decay": 0.0005,
          "momentum": 0.9,
          "nesterov": false
        }
      },
      "display_iter": 100,
      "save_iters": 5000,
      "test_interval": 5000,
      "max_epoch": 2
    },
    "res": {
      "nms": {
        "mode": "union",
        "max_threshold": 0.45,
        "pre_nms": 1000
      },
      "val_conf_thre": 0.01,
      "vis_conf_thre": 0.5,
      "max_per_image": 200,
      "cls_keep_num": 50
    },
    "loss": {
      "loss_type": "multibox_loss",
      "loss_weights": {
        "multibox_loss": {
            "multibox_loss": 1.0
        }
      }
    }
}'''
det_template_config_dict = json.loads(det_template_config)

In [54]:
det_config = det_template_config_dict

det_config['dataset'] = 'voc'
det_config['data']['data_dir'] = '/data/datasets/cv/VOC07+12_DET'
det_config['network']['model_name'] = 'vgg16_ssd300'

conf = json.dumps(det_config)

In [55]:
uuid = 'vgg300_ssd_voc'
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success",
    "message": "train.start task cache directory: /data/users/test/vgg300_ssd_voc"
}


In [50]:
data = '''{
    "op":"train.stop",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success"
}


# 评估